In [55]:
import torch
import torch.nn as nn
from torch.nn import Sequential
import torch.nn.functional as F
from torchvision import transforms, datasets, models
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
from collections import namedtuple
from IPython.display import Image
from torch.utils.data import Dataset, DataLoader
import os
from sklearn.preprocessing import normalize
from sklearn.metrics.pairwise import cosine_similarity
import random
from matplotlib import pyplot as plt
import pandas as pd
import time
from tqdm import tqdm
import csv


from torch.nn.utils import rnn
import pickle
from sklearn.metrics import accuracy_score
# from utils import Dataset, get_script_short_name
import time




TabError: inconsistent use of tabs and spaces in indentation (cnn_main.py, line 63)

In [2]:

print('Loading data...')
with open('../dataset/data_emb', 'rb') as f:
    all_sets, embs, word2idx = pickle.load(f)

Loading data...


In [3]:
# print (len(all_sets))
print (embs.shape)
# print(word2idx)

(62434, 300)


In [4]:
train_set = all_sets[0]
dev_set = all_sets[1]
test_set = all_sets[2]

In [15]:
train_y = np.array(train_set['y'])
dev_y = np.array(dev_set['y'])
test_y = np.array(test_set['y'])


In [22]:
train_q1 = np.array([np.array(seq) for seq in train_set['q1']])
train_q2 = np.array([np.array(seq) for seq in train_set['q2']])
dev_q1 = np.array([np.array(seq) for seq in dev_set['q1']])
dev_q2 = np.array([np.array(seq) for seq in dev_set['q2']])
test_q1 = np.array([np.array(seq) for seq in test_set['q1']])
test_q2 = np.array([np.array(seq) for seq in test_set['q2']])

In [24]:
print (train_q1[:2])

[array([ 2,  3,  4,  5,  6,  7,  8,  9,  2,  5,  4, 10, 11, 12, 13, 14, 15,
       16, 17])
 array([18, 19,  2, 20, 21, 22, 23, 17])]


In [34]:
# # save data sets to npz files
# # outfile_train = TemporaryFile()
# np.savez('train', train_q1, train_q2, train_y)
# # outfile_dev = TemporaryFile()
# np.savez('dev', dev_q1, dev_q2, dev_y)
# # outfile_test = TemporaryFile()
# np.savez('test', test_q1, test_q2, test_y)

In [41]:
# # load npz data sets
# train_set = np.load('train.npz')
# dev_set = np.load('dev.npz')
# test_set = np.load('test.npz')

# train_q1 = train_set['arr_0']
# train_q2 = train_set['arr_1']
# train_y = train_set['arr_2']
# dev_q1 = dev_set['arr_0']
# dev_q2 = dev_set['arr_1']
# dev_y = dev_set['arr_2']
# test_q1 = test_set['arr_0']
# test_q2 = test_set['arr_1']
# test_y = test_set['arr_2']


NameError: name 'Dataset' is not defined

In [28]:
print(train_q1[9].shape)

(15,)


In [14]:
print (all_sets[0])

KeyError: 0

In [22]:
idx2word = {value:key for key, value in word2idx.items()}

In [24]:
encode1 = all_sets[0]["q1"][0]
print (encode1)
question1 = [idx2word[idx] for idx in encode1]
print (question1)

[2, 3, 4, 5, 6, 7, 8, 9, 2, 5, 4, 10, 11, 12, 13, 14, 15, 16, 17]
['i', 'want', 'to', 'write', 'but', ',', 'what', 'should', 'i', 'write', 'to', 'improve', 'my', 'writing', 'ability', 'and', 'imaginative', 'power', '?']


In [30]:
# find max sentence length in the data
train_q1_max = max([len(q) for q in all_sets[0]["q1"]])
train_q2_max = max([len(q) for q in all_sets[0]["q2"]])
dev_q1_max = max([len(q) for q in all_sets[1]["q1"]])
dev_q2_max = max([len(q) for q in all_sets[1]["q2"]])
test_q1_max = max([len(q) for q in all_sets[2]["q1"]])
test_q2_max = max([len(q) for q in all_sets[2]["q2"]])

print(train_q1_max, train_q2_ma x, dev_q1_max, dev_q2_max, test_q1_max, test_q2_max)

# 146 271 89 271 81 130
# max word length in the entire dataset = 271

146 271 89 271 81 130


In [21]:
print (word2idx["apple"])
print (type(all_sets[0]["q1"]))
print (len(all_sets[0]["q1"]))
print (all_sets[0]["q1"][:100])


1681
<class 'list'>
323480
[[2, 3, 4, 5, 6, 7, 8, 9, 2, 5, 4, 10, 11, 12, 13, 14, 15, 16, 17], [18, 19, 2, 20, 21, 22, 23, 17], [24, 25, 26, 27, 28, 17], [8, 25, 29, 30, 31, 32, 33, 34, 32, 14, 32, 35, 32, 17, 8, 36, 2, 37, 38, 39, 17], [25, 40, 41, 42, 29, 43, 7, 44, 42, 29, 45, 40, 46, 47, 17, 8, 48, 49, 50, 17], [18, 19, 2, 51, 52, 53, 46, 54, 55, 56, 17], [57, 58, 59, 60, 61, 62, 63, 44, 64, 65, 66, 17], [8, 57, 29, 67, 68, 69, 70, 62, 71, 72, 14, 8, 57, 29, 73, 68, 74, 75, 76, 17], [77, 78, 79, 80, 81, 82, 17, 1, 77, 80, 81, 83, 17], [8, 57, 29, 84, 85, 86, 87, 88, 29, 89, 69, 90, 91, 92, 17], [8, 57, 93, 94, 45, 95, 9, 96, 97, 98, 99, 100, 101, 74, 1, 17], [8, 25, 29, 77, 14, 102, 103, 104, 105, 69, 29, 106, 17], [107, 108, 109, 110, 111, 4, 112, 17], [18, 19, 2, 113, 11, 114, 32, 115, 116, 17], [8, 25, 90, 117, 44, 118, 119, 65, 120, 17], [18, 19, 121, 122, 17], [59, 123, 25, 124, 44, 59, 125, 7, 14, 126, 19, 127, 96, 128, 129, 130, 126, 57, 131, 59, 132, 62, 133, 130, 134, 18, 